# APIM ❤️ OpenAI

## Private connectivity lab
![flow](./architecture.png)

Playground to show how to create a private network for consuming LLMs from `AI Services`.
This lab demonstrates how to create a private network for consuming LLMs from `AI Services` using `Private Link Services`, `Azure API Management (APIM)`, and `Azure Front Door`.

Notes:
- `Azure OpenAI` is only accessible through `Private Endpoints`. Public network access is disabled.
- `Azure API Management` is integrated in the private network and is used to manage the traffic to the `Azure OpenAI` service through `Private Endpoints`.
- `Azure API Management` is not accessible from a public network. The only access is through `Azure Front Door`.
- `Azure Front Door` manages the traffic to the `Azure API Management` service through `Private Link Service`.
- **`Azure Front Door` is the only publicly-accessible service.**

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [5]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-21-{deployment_name}" # change the name to match your naming style
resource_group_location = "francecentral"

apim_sku = 'Standardv2'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
openai_resources = [
    {"name": "openai1", "capacity": 20, "location": "eastus", "priority": 1},
    {"name": "openai2", "capacity": 20, "location": "swedencentral", "priority": 2, "weight": 50},
    {"name": "openai3", "capacity": 20, "location": "westus", "priority": 2, "weight": 50}
]

openai_deployment_name = "gpt-4o-mini"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_capacity = 8
openai_model_sku = 'Standard'
openai_api_version = "2024-02-01"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 21:47:20.385647 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [6]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 21:47:40.861929 :1s]
👉🏽 Current user: admin@MngEnvMCAP784683.onmicrosoft.com
👉🏽 Tenant ID: 93139d1e-a3c1-4d78-9ed5-878be090eba4
👉🏽 Subscription ID: dcef7009-6b94-4382-afdc-17eb160d709a


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

In [3]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "apimPublicNetworkAccess": { "value": "Enabled" },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters {bicep_parameters_file}",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-21-private-connectivity 
👉🏽 Resource group lab-21-private-connectivity does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-21-private-connectivity --location francecentral --tags source=ai-gateway 
✅ Resource group 'lab-21-private-connectivity' created ⌚ 21:04:19.536280 :5s]
{
  "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/lab-21-private-connectivity/providers/Microsoft.Resources/deployments/private-connectivity",
  "location": null,
  "name": "private-connectivity",
  "properties": {
    "correlationId": "7ac9f1d1-cc19-4c4f-9989-52f3ac4333fd",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/lab-21-private-connectivity/providers/Microsoft.Network/networkSecurityGroups/nsg-apim",
            "resourceGroup": "lab-21-private-connectivity",
            "re


d:\projects\Azure-Samples\AI-Gateway\labs\private-connectivity\main.bicep(355,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]




<a id='3'></a>
### 3️⃣ Approve Front Door private link connection to APIM

In the deployed Bicep template, Azure Front Door will establish a private link connection to the API Management service. This connection should be approved. Run the following command to approve the connection.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_resource_id = utils.get_deployment_output(output, 'apimResourceId', 'apimResourceId')

    outputPls = utils.run(f"az network private-endpoint-connection list --id {apim_resource_id} --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv")
    
    if outputPls.success:
        pls_connection_id = outputPls.text
        print(pls_connection_id)
        utils.run(f"az network private-endpoint-connection approve --id {pls_connection_id} --description 'Approved'", f"Private Link Connection approved.", f"Failed to approve Private Link Connection: {pls_connection_id}")

<a id='4'></a>
### 4️⃣ Disabling APIM public network access

As of May 2025, during the creation, the `APIM` service cannot disable the public network access. This behavior might change in the future. As a workaround, you can disable the public network access after the deployment is completed. To do that, you can run the Bicep template again after changing the `apimPublicNetworkAccess` parameter to `Disabled`. This will update the APIM service and disable the public network access.

In [ ]:
bicep_parameters['parameters']['apimPublicNetworkAccess']['value'] = "Disabled"

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='5'></a>
### 5️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [7]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    frontdoor_endpoint = utils.get_deployment_output(output, 'frontDoorEndpointHostName', 'Front Door Endpoint')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)

⚙️ Running: az deployment group show --name private-connectivity -g lab-21-private-connectivity 
✅ Retrieved deployment: private-connectivity ⌚ 22:11:52.210747 :4s]
👉🏽 Front Door Endpoint: afd-ppgufaienvvcm-g5fzc4b0bed8bre5.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-6ac5zecx4zzjk.azure-api.net
👉🏽 APIM Subscription Key (masked): ****7e20


<a id='6'></a>
### 6️⃣ 🧪 Test the API using a direct HTTP call through Frontdoor

Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

In [8]:
import requests

def call_api(resource_url):
    url = f"{resource_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"

    messages = {"messages": [
        {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)

    print(f"Response status code: {response.status_code}")

    if (response.status_code == 200):
        data = json.loads(response.text)
        print(f"💬 {data.get('choices')[0].get('message').get('content')}\n")
    else:
        print(f"{response.text}\n")

call_api(f"https://{frontdoor_endpoint}")

Response status code: 200
💬 Oh sure! Just give me a moment to check my non-existent watch. But really, why not just look at your phone? It’s probably glued to your hand anyway!



<a id='7'></a>
### 7️⃣ 🧪 Test APIM API access through public network

APIM is not accessible from public network. The only way to access it is through Azure Front Door. This test should fail with a 403 error.

In [9]:
call_api(apim_resource_gateway_url)

Response status code: 403
{ "statusCode": 403, "message": "Request originated from client public IP address 74.241.135.213, public network access on this `Microsoft.ApiManagement/service/apim-6ac5zecx4zzjk` is disabled. To connect to `Microsoft.ApiManagement/service/apim-6ac5zecx4zzjk`, please use the Private Endpoint from inside your virtual network. To learn more https://aka.ms/apim-privateendpoint " }



<a id='8'></a>
### 8 🧪 Test APIM API access through private virtual network

APIM is accessible in the private virtual network through a `Private Endpoint`. This means that any resource within this network can communicate securely with the APIM service. To validate this behavior, the Bicep template creates an `Azure virtual machine` that will act as a `jumpbox` and a `Bastion Host` that will provide secure communication with the VM through `ssh`. 

Call the APIM API from within the VM. Navigate to the VM and connect to it using `Bastion` using the default username `azureuser` and password `@Aa123456789`.

The following code will generate a `cUrl` command that will test the API access. Copy its output and run it in the VM. 

This test should succeed with a 200 OK response.

In [10]:
print(f'''curl -X POST "{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}" \\
  -H "Content-Type: application/json" \\
  -H "api-key: {apim_subscription_key}" \\
  -d '{{"messages": [ 
        {{"role": "system", "content": "You are a helpful assistant."}}, 
        {{"role": "user", "content": "What are 3 things to visit in Seattle?"}} 
      ]}}'
''')

curl -X POST "https://apim-6ac5zecx4zzjk.azure-api.net/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-02-01" \
  -H "Content-Type: application/json" \
  -H "api-key: fc8ef9938c6a4bda88817394ba5f7e20" \
  -d '{"messages": [ 
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "What are 3 things to visit in Seattle?"} 
      ]}'



<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.